### 3D City Models from Volunteered Public Data

**This notebook will produce an interactive 3D Building Model visualization** *- via [pydeck](https://deckgl.readthedocs.io/en/latest/) -* **which you can navigate and query.**

In [48]:
#load the magic

%matplotlib inline
import os
from pathlib import Path
import requests

import overpass
import osm2geojson

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, shape, mapping
import json
import geojson

from osgeo import ogr

import pydeck as pdk

**Harvest [OpenStreetMap](https://en.wikipedia.org/wiki/OpenStreetMap)** - Query the [Overpass API](https://wiki.openstreetmap.org/wiki/Overpass_API) from within Jupyter and convert to .geojson.

<div class="alert alert-block alert-info"><b>Set an area-of-interest:</b> 
    
This is done: larger area -> focus area or State (Province) -> Suburb (Village)
</div>

In [49]:
large = 'Western Cape'
focus = 'Cape Peninsula University of Technology (Bellville Campus)'

In [52]:
query = """
[out:json][timeout:30];
// --when areas have duplicate names given the world has a limited amount of uniquely named places
// --main area
area[name='{0}']->.b;
// -- target area ~ can be way or relation
way(area.b)[name='{1}'];
map_to_area -> .a;
(
  (
    // I want all buildings
    way[building](area.a);

    // plus every building:part
    way["building:part"](area.a);
    // and relation type=multipolygon ~ to removed courtyards from buildings
    relation["building"]["type"="multipolygon"](area.a);
  );
-
  // excluding buildings with relation type=building role=outline
  // to remove outlines that surround building:part
  (
    // for every way in the input set select the relations of which it is an "outline" member
    rel(bw:"outline")["type"="building"];
    // back to the ways with role "outline"
    way(r:"outline");
  );
);
out body;
>;
out skel qt;
""".format(large, focus)

url = "http://overpass-api.de/api/interpreter"
r = requests.get(url, params={'data': query})
#rr = r.read()
gj = osm2geojson.json2geojson(r.json())

In [53]:
# have a look at a random feature
gj['features'][1]

{'type': 'Feature',
 'properties': {'type': 'way',
  'id': 897473388,
  'tags': {'addr:city': 'Cape Town',
   'addr:postcode': '7530',
   'addr:suburb': 'Bellville',
   'building:levels': '3',
   'building:part': 'university',
   'name': 'Construction Engineering'}},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[18.6418348, -33.9317775],
    [18.6418213, -33.9317012],
    [18.6417798, -33.9314661],
    [18.6417665, -33.9313906],
    [18.6417627, -33.9313689],
    [18.6419021, -33.9313522],
    [18.6418983, -33.93133],
    [18.6420934, -33.9313066],
    [18.6420968, -33.9313256],
    [18.6422393, -33.9313085],
    [18.6422441, -33.9313359],
    [18.6423151, -33.9317262],
    [18.6418348, -33.9317775]]]}}

<div class="alert alert-block alert-warning"><b>Calculate building height:</b> 

We assume a building level is 2.8 meters high and add another 1.3 meters (to account for the roof) and create a new attribute height.</div>

In [ ]:
#with open('./data/fp.geojson') as f:
    #gj = geojson.load(f)

In [54]:
storeyheight = 2.8

#-- iterate through the list of buildings and create GeoJSON features rich in attributes
footprints = {
    "type": "FeatureCollection",
    "features": []
    }

for i in gj['features']:
    f = {
    "type" : "Feature"
    }
    # at a minimum we only want building:levels tagged
    if 'building:levels' in i['properties']['tags']:
        f["properties"] = {}
        
        for p in i["properties"]:
        #print(i["properties"]['tags']['building:levels'])
        #break
        #-- store all OSM attributes and prefix them with osm_
            f["properties"]["osm_%s" % p] = i["properties"][p]
            osm_shape = shape(i["geometry"])
            #-- a few buildings are not polygons, rather linestrings. This converts them to polygons
            #-- rare, but if not done it breaks the code later
            if osm_shape.type == 'LineString':
                osm_shape = Polygon(osm_shape)
                #-- and multipolygons must be accounted for
            elif osm_shape.type == 'MultiPolygon':
                osm_shape = Polygon(osm_shape[0])
                #-- convert the shapely object to geojson
            f["geometry"] = mapping(osm_shape)

    #-- finally calculate the height and store it as an attribute (extrusion of geometry 
    ## -- will be done in the next script)
            f["properties"]['height'] = float(i["properties"]['tags']['building:levels']) * storeyheight + 1.3    
            footprints['features'].append(f)

#-- store the data as GeoJSON
with open('./data/fp.geojson', 'w') as outfile:
    json.dump(footprints, outfile)

**[pydeck](https://deckgl.readthedocs.io/en/latest/) needs an area and a center**, harvest from osm as well.

In [55]:
query = """
[out:json][timeout:30];
area[name='{0}']->.a;
//gather results
(
  // query part for: “university”
  way['name'='{1}'](area.a);
);
//print results
out body;
>;
out skel qt;
""".format(large, focus)

url = "http://overpass-api.de/api/interpreter"
r = requests.get(url, params={'data': query})
#rr = r.read()
area = osm2geojson.json2geojson(r.json())

In [56]:
gdf = gpd.GeoDataFrame.from_features(area['features'])

bounds = gdf.geometry.bounds
x = gdf.centroid.x
y = gdf.centroid.y

bbox = [(bounds.minx, bounds.miny), (bounds.minx, bounds.maxy), 
        (bounds.maxx, bounds.maxy), (bounds.maxx, bounds.miny)]

***~ In order to make the most of the semantic data we need to extract the `osm_tags` from the dictionary: and add it as `tooltips` to the visualization.***

In [57]:
data = './data/fp.geojson'
json = pd.read_json(data)
build_df = pd.DataFrame()

# Parse the geometry out to Pandas
build_df["coordinates"] = json["features"].apply(lambda row: row["geometry"]["coordinates"])
build_df["height"] = round(json["features"].apply(lambda row: row["properties"]["height"]), 1)
#build_df["tags"] = json["features"].apply(lambda row: row["properties"]["osm_tags"])

#we want to display data so extract values from the dictionary 
build_df["tags"] = json["features"].apply(lambda row: row["properties"]["osm_tags"])
build_df['level'] = build_df['tags'].apply(lambda x: x.get('building:levels'))
build_df['name'] = build_df['tags'].apply(lambda x: x.get('name'))

In [34]:
build_df.head(2)

,coordinates,height,tags,level,city,street,type
0,"[[[18.642799, -33.931231], [18.642878, -33.931...",9.7,"{'addr:city': 'Cape Town', 'addr:postcode': '7...",3,Cape Town,None,None
1,"[[[18.642799, -33.931231], [18.642878, -33.931...",9.7,"{'addr:city': 'Cape Town', 'addr:postcode': '7...",3,Cape Town,None,None


*I want additional visual effects to show the potential and power of **3D City Models**; namely: parks, agricultural land and waterways (streams). We get this from OpenStreetMap as well.*

In [58]:
query = """
[out:json][timeout:25];
area[name='Cape Peninsula University of Technology (Bellville Campus)'];
(
  // query
  way["leisure"~'pitch'](area);
);
// print results
out body;
>;
out skel qt;
"""

url = "http://overpass-api.de/api/interpreter"
p = requests.get(url, params={'data': query})
#rr = r.read()
green_spaces = osm2geojson.json2geojson(p.json())

query = """
[out:json][timeout:25];
area[name='Cape Peninsula University of Technology (Bellville Campus)'];
(
  // query
  way["natural"~'water'](area);
);
// print results
out body;
>;
out skel qt;
"""

url = "http://overpass-api.de/api/interpreter"
w = requests.get(url, params={'data': query})
#rr = r.read()
water_spaces = osm2geojson.json2geojson(w.json())

query = """
[out:json][timeout:25];
area[name='Cape Peninsula University of Technology (Bellville Campus)'];
(
  // query
  way['parking'](area);
);
// print results
out body;
>;
out skel qt;
"""

url = "http://overpass-api.de/api/interpreter"
p = requests.get(url, params={'data': query})
#rr = r.read()
p_spaces = osm2geojson.json2geojson(p.json())

In [59]:
## ~ (x, y) - bl, tl, tr, br  ~~ or ~~ sw, nw, ne, se
#area = [[[18.4377, -33.9307], [18.4377, -33.9283], [18.4418, -33.9283], [18.4418, -33.9307]]]
area = [[[bbox[0][0][0], bbox[0][1][0]], [bbox[1][0][0], bbox[1][1][0]], 
         [bbox[2][0][0], bbox[2][1][0]], [bbox[3][0][0], bbox[3][1][0]]]]

## ~ (y, x)
view_state = pdk.ViewState(latitude=y[0], longitude=x[0], zoom=16.5, max_zoom=19, pitch=72, 
                                   bearing=80)

land = pdk.Layer(
    "PolygonLayer",
    area,
    stroked=False,
    # processes the data as a flat longitude-latitude pair
    get_polygon="-",
    get_fill_color=[0, 0, 0, 1],
    #material = True,
    #shadowEnabled = True
)
building_layer = pdk.Layer(
    "PolygonLayer",
    build_df,
    #id="geojson",
    opacity=0.3,
    stroked=False,
    get_polygon="coordinates",
    filled=True,
    extruded=True,
    wireframe=False,
    get_elevation="height",
    get_fill_color="[255, 255, 255]", #255, 255, 255
    get_line_color=[255, 255, 255],
    #material = True, 
    #shadowEnabled = True, 
    auto_highlight=True,
    pickable=True,
)
greenspaces_layer =  pdk.Layer(
    "GeoJsonLayer",
    green_spaces,
    opacity=0.5,
    stroked=False,
    filled=True,
    wireframe=True,
    get_fill_color="[14, 140, 58]",
    get_line_color='[14, 140, 58]',
)
p_layer =  pdk.Layer(
    "GeoJsonLayer",
    p_spaces,
    opacity=0.2,
    stroked=False,
    filled=True,
    wireframe=True,
    get_fill_color="[128,128,128]",
    get_line_color='[128,128,128]',
)

water_layer =  pdk.Layer(
    "GeoJsonLayer",
    water_spaces,
    opacity=0.8,
    stroked=False,
    filled=True,
    wireframe=True,
    get_fill_color="[35, 35, 142]",
    get_line_color='[35, 35, 142]',
)
tooltip = {"html": "<b>Levels:</b> {level} <br/> <b>Name:</b> {name}"}

r = pdk.Deck(layers=[land, building_layer, greenspaces_layer, p_layer, water_layer], 
             #views=[{"@@type": "MapView", "controller": True}],
             initial_view_state=view_state,
             map_style = 'dark_no_labels', 
             tooltip=tooltip)
#save
r.to_html("./result/interactiveOnly.html")

**on a laptop without a mouse:**

- `trackpad left-click drag-left` and `-right`;
- `Ctrl left-click drag-up`, `-down`, `-left` and `-right` to rotate and so-on and
- `+` next to Backspace zoom-in and `-` next to `+` zoom-out.

[osm_lod1_3dbuildingmodel.ipynb](https://github.com/AdrianKriger/osm_LoD1_3Dbuildings/blob/main/osm_lod1_3dbuildingmodel.ipynb) offers tips to add features specific to a residential area.

**Now you do your community.** ~ If your area lacks OpenStreetMap data and you want to contribute please follow the [Guide](https://wiki.openstreetmap.org/wiki/Beginners%27_guide).